In [1]:
import pandas as pd

In [32]:
# Carga de datos
match_df = pd.read_pickle('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\match_data_version1.pickle')
winner_df =  pd.read_pickle('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\match_loser_data_version1.pickle')
loser_df = pd.read_pickle('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\match_winner_data_version1.pickle')

In [33]:
# Informacion de los dataframes
match_df.info()
winner_df.info()
loser_df.info()
match_df['participants'].iloc[0][0]['timeline']

<class 'pandas.core.frame.DataFrame'>
Index: 108829 entries, 0 to 0
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   gameCreation           108829 non-null  float64
 1   gameDuration           108829 non-null  float64
 2   gameId                 108829 non-null  float64
 3   gameMode               108829 non-null  object 
 4   gameType               108829 non-null  object 
 5   gameVersion            108829 non-null  object 
 6   mapId                  108829 non-null  float64
 7   participantIdentities  108829 non-null  object 
 8   participants           108829 non-null  object 
 9   platformId             108829 non-null  object 
 10  queueId                108829 non-null  float64
 11  seasonId               108829 non-null  float64
 12  status.message         0 non-null       object 
 13  status.status_code     0 non-null       float64
dtypes: float64(7), object(7)
memory usage: 12.5+ M

{'participantId': 1,
 'creepsPerMinDeltas': {'10-20': 6.199999999999999, '0-10': 6.1},
 'xpPerMinDeltas': {'10-20': 473.5, '0-10': 386.6},
 'goldPerMinDeltas': {'10-20': 277, '0-10': 315.6},
 'damageTakenPerMinDeltas': {'10-20': 906, '0-10': 473.79999999999995},
 'role': 'DUO_CARRY',
 'lane': 'MIDDLE'}

In [34]:
# First, we need to drop gameCreation, gameType, gameVersion, mapId, platformId, queueId, seasonId
# status.message, status.status_code
# since these are either unique value column or null or irrelevant to our app.
# It will help us to reduce the size of the file and be able to process the data furether locally
match_df.drop(['gameCreation', 'gameType', 'gameVersion', 'mapId',
                    'platformId', 'queueId', 'seasonId', 'status.message', 'status.status_code'], axis=1, inplace=True)

In [35]:
# Second, we need to remove the games data that are not CLASSIC gameMode
# since we are focusing of classic mode game (90% of the data)
indexList = [i for i in match_df[match_df.gameMode != 'CLASSIC'].index if i != 0]

print('Dropping non-classic games...')
match_df.drop(indexList, inplace=True)
match_df.drop(['gameMode'], axis=1, inplace=True)

print("Succeed!")

Dropping non-classic games...
Succeed!


In [37]:
# Remove unused data from participants:
# participants: spell1Id, spell2Id (no table to specify other info)
#               stats:    (no table to specify other info, or irrelevant)
#                         combatPlayerScore, objectivePlayerScore, totalPlayerScore
#                         totalScoreRank, playerScore0, playerScore1, playerScore2
#                         playerScore3, playerScore4, playerScore5, playerScore6
#                         playerScore7, playerScore8, playerScore9, perk0
#                         perk0Var1, perk0Var2, perk0Var3, perk1, perk1Var1
#                         perk1Var2, perk1Var3, perk2, perk2Var1, perk2Var2,
#                         perk2Var3, perk3, perk3Var1, perk3Var2, perk3Var3,
#                         perk4, perk4Var1, perk4Var2, perk4Var3, perk5,
#                         perk5Var1, perk5Var2, perk5Var3, perkPrimaryStyle,
#                         perkSubStyle, statPerk0, statPerk1, statPerk2
#               timeline: we are going to extract the 'role', 'lane' from timeline
#                         and put them inside 'participants' directly
entriesToRemoveMain = ['spell1Id', 'spell2Id']
entriesToRemoveStats = ['combatPlayerScore', 'objectivePlayerScore', 'totalPlayerScore',
                        'totalScoreRank', 'playerScore0', 'playerScore1', 'playerScore2',
                        'playerScore3', 'playerScore4', 'playerScore5', 'playerScore6',
                        'playerScore7', 'playerScore8', 'playerScore9', 'perk0',
                        'perk0Var1', 'perk0Var2', 'perk0Var3', 'perk1', 'perk1Var1',
                        'perk1Var2', 'perk1Var3', 'perk2', 'perk2Var1', 'perk2Var2',
                        'perk2Var3', 'perk3', 'perk3Var1', 'perk3Var2', 'perk3Var3',
                        'perk4', 'perk4Var1', 'perk4Var2', 'perk4Var3', 'perk5',
                        'perk5Var1', 'perk5Var2', 'perk5Var3', 'perkPrimaryStyle',
                        'perkSubStyle', 'statPerk0', 'statPerk1', 'statPerk2']

def deleteKeysParticipants(participantsData):
    for participant in participantsData:
        for mainKey in entriesToRemoveMain:
            participant.pop(mainKey, None)
        for statsKey in entriesToRemoveStats:
            try:
                participant['stats'].pop(statsKey, None)
            except:
                # During the data diving, we found that some participant does not have 'stats',
                # which is resulted by disconnection, we will delete the this participant since
                # it is useless
                participantsData = [p for p in participantsData if p != participant]
                break
        # Extraction
        roleValue = participant['timeline']['role']
        laneValue = participant['timeline']['lane']
        participant.pop('timeline', None)
        participant['role'] = roleValue
        participant['lane'] = laneValue

    return participantsData

match_df['participants'].apply(lambda x: deleteKeysParticipants(x))

print("Succeed!")

Succeed!


In [38]:
# There are some data in participantIdentities missing summonerId
# We will get rid of these rows
def is_summoner_id_misssing(participantsData):
    for participant in participantsData:
        if 'summonerId' not in participant['player']:
            return False
    return True

match_df = match_df[match_df.apply(lambda row: is_summoner_id_misssing(row['participantIdentities']), axis=1)]

In [39]:
# There are some games that the participantIdentities or participants are not equal to 10 (LOL classical games must have 10 player)
# We will get rid of these data
def is_number_participant_identities_ten(participantsData, gamID):
    if len(participantsData) != 10:
        return False
    return True

def is_number_participant_ten(participantsData):
    if len(participantsData) != 10:
        return False
    return True

match_df = match_df[match_df.apply(lambda row: is_number_participant_identities_ten(row['participantIdentities'], row['gameId']), axis=1)]
match_df = match_df[match_df.apply(lambda row: is_number_participant_ten(row['participants']), axis=1)]

In [40]:
# A player can have multiple summoners under the same account, and each summoner has its own summonerId.
# However, each player account has only one accountId.
# Since we have summoner name and do not have account name in the dataset,
# we expect the potential user for our application to use summonerName property to identify itself.

# Remove unused data from participantIdentities:
# player:
#        platformId (Dataset is Korea only)
#        accountId
#        currentPlatformId (Dataset is Korea only)
#        currentAccountId
#        matchHistoryUri
#        profileIcon
entriesToRemovePlayer = ['platformId', 'accountId', 'currentPlatformId',
                         'currentAccountId', 'matchHistoryUri', 'profileIcon']

def deleteKeysParticipantIdentities(participantsData):
    for participant in participantsData:
        for playerKey in entriesToRemovePlayer:
            participant['player'].pop(playerKey, None)
    return participantsData

match_df['participantIdentities'].apply(lambda x: deleteKeysParticipantIdentities(x))

print("Succeed!")

Succeed!


In [41]:
# Finally, drop duplicated gameId rows
match_df = match_df.drop_duplicates(subset=['gameId'])

In [42]:
# Note, we will get rid of the index when saving to csv file
match_df.head()

,gameDuration,gameId,participantIdentities,participants
0,1323.0,4.247263e+09,"[{'participantId': 1, 'player': {'summonerName...","[{'participantId': 1, 'teamId': 100, 'champion..."
1,1317.0,4.247156e+09,"[{'participantId': 1, 'player': {'summonerName...","[{'participantId': 1, 'teamId': 100, 'champion..."
2,932.0,4.243963e+09,"[{'participantId': 1, 'player': {'summonerName...","[{'participantId': 1, 'teamId': 100, 'champion..."
3,2098.0,4.241678e+09,"[{'participantId': 1, 'player': {'summonerName...","[{'participantId': 1, 'teamId': 100, 'champion..."
4,2344.0,4.241539e+09,"[{'participantId': 1, 'player': {'summonerName...","[{'participantId': 1, 'teamId': 100, 'champion..."


In [43]:
# Participant data in participants list per game
match_df['participants'].iloc[0][0]

{'participantId': 1,
 'teamId': 100,
 'championId': 7,
 'stats': {'participantId': 1,
  'win': False,
  'item0': 2033,
  'item1': 3285,
  'item2': 3108,
  'item3': 1082,
  'item4': 3020,
  'item5': 2055,
  'item6': 3364,
  'kills': 3,
  'deaths': 6,
  'assists': 3,
  'largestKillingSpree': 2,
  'largestMultiKill': 1,
  'killingSprees': 1,
  'longestTimeSpentLiving': 564,
  'doubleKills': 0,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 71369,
  'magicDamageDealt': 56332,
  'physicalDamageDealt': 13037,
  'trueDamageDealt': 1999,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 10844,
  'magicDamageDealtToChampions': 8926,
  'physicalDamageDealtToChampions': 1918,
  'trueDamageDealtToChampions': 0,
  'totalHeal': 4006,
  'totalUnitsHealed': 1,
  'damageSelfMitigated': 6742,
  'damageDealtToObjectives': 1055,
  'damageDealtToTurrets': 1008,
  'visionScore': 14,
  'timeCCingOthers': 12,
  'totalDamageTaken': 17255,
  'ma

In [44]:
# ParticipantIdentities data in participantIdentities list per game
match_df['participantIdentities'].iloc[0][0]

{'participantId': 1,
 'player': {'summonerName': '쪼렙이다말로하자',
  'summonerId': 'YTMZt7CQ7cHXiADp1ZtI1GjNHdAE_HQbbSxxq_gllWinqQ'}}

In [46]:
match_df.to_csv('matchData.csv', index=False, encoding='utf-8')

In [47]:
chunk_size = 20000
def write_chunk(part, lines):
    with open('matchDataPart'+ str(part) +'.csv', 'w') as f_out:
        f_out.write(header)
        f_out.writelines(lines)

with open("C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\matchData.csv", "r") as f:
    count = 0
    header = f.readline()
    lines = []
    for line in f:
        count += 1
        lines.append(line)
        if count % chunk_size == 0:
            write_chunk(count // chunk_size, lines)
            lines = []
    # write remainder
    if len(lines) > 0:
        write_chunk((count // chunk_size) + 1, lines)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 134: character maps to <undefined>

In [48]:
# Load the newest version of riot_champion csv file
champion_data_df = pd.read_csv('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\riot_champion.csv')

In [49]:
# First, we need to drop 'Unnamed: 0'(Unnamed index column), version, 'image.full', 'image.sprite',
# 'image.group', 'image.x', 'image.y', 'image.w', 'image.h'
# since these are either unique value column or null or irrelevant to our app.
champion_data_df.drop(['Unnamed: 0', 'version', 'image.full', 'image.sprite',
                       'image.group', 'image.x', 'image.y', 'image.w', 'image.h',
                       'stats.crit', 'stats.critperlevel'], axis=1, inplace=True)

In [50]:
# Second, we rename some of the columns so that the column names are more trival.
champion_data_df.rename(columns={'info.attack' : 'baseAttack', 'info.defense' : 'baseDefense',
                                 'info.magic' : 'baseMagic', 'info.difficulty' : 'difficulty',
                                 'stats.hp' : 'hp', 'stats.hpperlevel' : 'hpPerLevel',
                                 'stats.mp' : 'mp', 'stats.mpperlevel' : 'mpPerLevel',
                                 'stats.movespeed' : 'moveSpeed', 'stats.armor' : 'armor',
                                 'stats.armorperlevel' : 'armorPerLevel', 'stats.spellblock' : 'spellBlock',
                                 'stats.spellblockperlevel' : 'spellBlockPerLevel', 'stats.attackrange' : 'attackRange',
                                 'stats.hpregen' : 'hpRegen', 'stats.hpregenperlevel' : 'hpRegenPerLevel',
                                 'stats.mpregen' : 'mpRegen', 'stats.mpregenperlevel' : 'mpRegenPerLevel',
                                 'stats.attackdamage' : 'attackDamage', 'stats.attackdamageperlevel' : 'attackDamagePerLevel',
                                 'stats.attackspeed' : 'attackSpeed', 'stats.attackspeedperlevel' : 'attackSpeedPerLevel'}, inplace=True)

In [51]:
champion_data_df.to_csv('championData.csv', index=False, encoding='utf-8')

In [52]:
# Load the newest version of riot_item.csv csv file
item_data_df = pd.read_csv('C:\\Users\\Facu\\PycharmProjects\\Proyecto_Final_IA\\proyecto_final\\data\\riot_item.csv')

In [53]:
# First, we need to drop 'Unnamed: 0'(Unnamed index column)
item_data_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [54]:
# Second, we need to rename some of the columns for this dataset.
item_data_df.rename(columns={'item_id' : 'itemId', 'upper_item' : 'upperItem',
                             'buy_price' : 'buyPrice', 'sell_price' : 'sellPrice'}, inplace=True)

In [55]:
item_data_df.to_csv('itemData.csv', index=False, encoding='utf-8')